# Instalação de dependências

In [1]:
%pip install pandas pyarrow tdqm requests ipywidgets huggingface_hub

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1384 sha256=c8b1d9c9205d40e16c310b50ad69e1950cd441710b9ce92dfba76bd27949fbf2
  Stored in directory: /Users/felipe/Library/Caches/pip/wheels/af/02/71/aae0f7ee738abf19498353918ddae0f90a0d6ceb337b0bbc91
Successfully built tdqm
Note: you may need to restart the kernel to use updated packages.


# Importa dependências

In [2]:
import os
from pathlib import Path
from datetime import datetime
import requests
import zipfile
import concurrent.futures
import pandas as pd
from functools import partial
from tqdm.auto import tqdm
import ipywidgets as widgets
from IPython.display import display
import time

# Download do dataset da Binance

In [3]:
# Função para gerar todos os meses entre as datas de início e fim
def generate_months(start_date, end_date):
    current = start_date
    while current <= end_date:
        yield current
        # Avança para o próximo mês
        if current.month == 12:
            current = datetime(current.year + 1, 1, 1)
        else:
            current = datetime(current.year, current.month + 1, 1)

# Função para baixar um arquivo com atualização da barra de progresso e velocidade
def download_file(url, dest_path, progress_bar, speed_label):
    try:
        with requests.get(url, stream=True) as response:
            response.raise_for_status()  # Levanta uma exceção para erros HTTP
            total_size = int(response.headers.get('content-length', 0))
            block_size = 1024  # 1 Kibibyte
            downloaded = 0
            start_time = time.time()
            with open(dest_path, 'wb') as f:
                for data in response.iter_content(block_size):
                    if data:
                        f.write(data)
                        downloaded += len(data)
                        progress_bar.value = downloaded
                        elapsed_time = time.time() - start_time
                        speed = downloaded / elapsed_time if elapsed_time > 0 else 0
                        speed_label.value = f"Velocidade: {speed/1024:.2f} KB/s"
        progress_bar.description = f"✅ {dest_path.name}"
    except requests.exceptions.HTTPError as http_err:
        progress_bar.description = f"❌ {dest_path.name}"
        speed_label.value = f"Erro HTTP: {http_err}"
    except Exception as err:
        progress_bar.description = f"❌ {dest_path.name}"
        speed_label.value = f"Erro: {err}"

# Definição das datas de início e fim
start_date = datetime(2024, 10, 1)   # Agosto de 2017
end_date = datetime(2024, 12, 1)   # Setembro de 2024

# Padrão da URL base
base_url = "https://data.binance.vision/data/spot/monthly/trades/BTCUSDT/BTCUSDT-trades-{year}-{month:02d}.zip"

# Diretório de download
download_dir = Path("./dataset-raw")
download_dir.mkdir(parents=True, exist_ok=True)

# Lista para armazenar as tarefas de download
download_tasks = []
for single_date in generate_months(start_date, end_date):
    year = single_date.year
    month = single_date.month
    file_suffix = f"{year}-{month:02d}"

    # Caminhos dos arquivos
    csv_file = download_dir / f"BTCUSDT-trades-{file_suffix}.csv"
    zip_file = download_dir / f"BTCUSDT-trades-{file_suffix}.zip"

    # Verifica se o CSV já existe
    if csv_file.exists():
        print(f"📄 CSV já existe: {csv_file.name}. Pulando download.")
        continue
    # Verifica se o ZIP já existe
    elif zip_file.exists():
        print(f"📦 ZIP já existe: {zip_file.name}. Pulando download.")
        continue
    else:
        # Constrói a URL de download
        url = base_url.format(year=year, month=month)
        download_tasks.append((url, zip_file))

# Número máximo de threads
max_workers = 1  # Ajuste conforme a capacidade do seu sistema e conexão de rede

# Função principal para gerenciar os downloads
def main_download(download_tasks, max_workers=5):
    if not download_tasks:
        print("✅ Nenhuma tarefa de download a ser executada.")
        return

    # Criação de widgets para cada download
    download_widgets = []
    for url, dest_path in download_tasks:
        speed_label = widgets.Label(value="Velocidade: 0 KB/s")
        progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=1,  # Será atualizado após obter o tamanho total
            description=dest_path.name,
            bar_style='',  # 'success', 'info', 'warning', 'danger' ou ''
            orientation='horizontal'
        )
        download_widgets.append(widgets.VBox([progress_bar, speed_label]))

    # Exibe todos os widgets de download
    container = widgets.VBox(download_widgets)
    display(container)

    # Inicializa o ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submete todas as tarefas de download
        futures = []
        for i, (url, dest_path) in enumerate(download_tasks):
            # Atualiza o valor máximo da barra de progresso após obter o tamanho total
            try:
                head = requests.head(url, allow_redirects=True)
                total_size = int(head.headers.get('content-length', 0))
                if total_size == 0:
                    # Fallback para obter o tamanho via GET
                    with requests.get(url, stream=True) as response:
                        response.raise_for_status()
                        total_size = int(response.headers.get('content-length', 0))
            except Exception as e:
                print(f"❌ Não foi possível obter o tamanho de {dest_path.name}: {e}")
                continue

            progress_bar = download_widgets[i].children[0]
            speed_label = download_widgets[i].children[1]
            progress_bar.max = total_size

            future = executor.submit(download_file, url, dest_path, progress_bar, speed_label)
            futures.append(future)

        # Aguardando a conclusão de todas as tarefas
        for future in concurrent.futures.as_completed(futures):
            pass  # Todas as atualizações são feitas dentro da função download_file

    print("🎉 Todos os downloads foram concluídos.")

# Executa a função de download
main_download(download_tasks, max_workers)


🎉 Todos os downloads foram concluídos.


# Processar dataset de .zip para .parquet

In [4]:
# Define os nomes das colunas com base na estrutura dos dados
COLUMN_NAMES = [
    'trade_id',
    'price',
    'qty',
    'quoteQty',
    'time',
    'isBuyerMaker',
    'isBestMatch'
]

import os
import zipfile
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

def extract_zip(file_path, extract_to):
    """
    Extrai um arquivo ZIP e o remove após a extração bem-sucedida.

    :param file_path: Caminho completo para o arquivo ZIP.
    :param extract_to: Diretório onde os arquivos serão extraídos.
    """
    try:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        os.remove(file_path)
        print(f"Extração concluída: {os.path.basename(file_path)}")
    except zipfile.BadZipFile:
        print(f"Arquivo corrompido: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"Erro ao processar {os.path.basename(file_path)}: {e}")

def extract_and_delete(zip_dir_path):
    """
    Extrai todos os arquivos ZIP em um diretório e os remove após a extração.

    :param zip_dir_path: Diretório contendo os arquivos ZIP.
    """
    # Lista todos os arquivos ZIP no diretório especificado
    zip_files = [
        os.path.join(zip_dir_path, f)
        for f in os.listdir(zip_dir_path)
        if f.lower().endswith('.zip')
    ]

    if not zip_files:
        print("Nenhum arquivo ZIP encontrado para extração.")
        return

    print(f"Iniciando a extração de {len(zip_files)} arquivos ZIP...")

    for zip_file in zip_files:
        extract_zip(zip_file, zip_dir_path)

    print("Extração de arquivos ZIP concluída.")

def process_csv_directory(directory_path, output_parquet_path):
    """
    Processa todos os arquivos CSV em um diretório e salva os dados combinados em um único arquivo Parquet,
    respeitando o limite de memória disponível.

    :param directory_path: Diretório contendo os arquivos CSV.
    :param output_parquet_path: Caminho onde o arquivo Parquet será salvo.
    """
    csv_files = [
        os.path.join(directory_path, f)
        for f in sorted(os.listdir(directory_path))
        if f.lower().endswith('.csv')
    ]

    if not csv_files:
        print("Nenhum arquivo CSV encontrado para processamento.")
        return

    print(f"Iniciando o processamento de {len(csv_files)} arquivos CSV...")

    # Inicializa o ParquetWriter
    writer = None

    for idx, file_path in enumerate(csv_files):
        try:
            # Ler o CSV em chunks para economizar memória
            for df_chunk in pd.read_csv(file_path, header=None, names=COLUMN_NAMES, chunksize=100000):
                # Converte 'time' de milissegundos para datetime
                df_chunk['time'] = pd.to_datetime(df_chunk['time'], unit='ms')
                table = pa.Table.from_pandas(df_chunk)
                if writer is None:
                    # Cria o ParquetWriter no primeiro chunk
                    writer = pq.ParquetWriter(output_parquet_path, table.schema, compression='snappy')
                writer.write_table(table)
            print(f"Dados do arquivo {os.path.basename(file_path)} processados.")
        except Exception as e:
            print(f"Erro ao processar o arquivo {os.path.basename(file_path)}: {e}")

    # Fecha o ParquetWriter
    if writer:
        writer.close()
        print(f"Dados combinados salvos com sucesso em {output_parquet_path}")
    else:
        print("Nenhum dado foi escrito no arquivo Parquet.")

    print("Processamento concluído.")


In [7]:
# Especifica o caminho para o diretório contendo os arquivos ZIP e CSV
zip_dir_path = './dataset-raw'  # Substitua pelo seu caminho de diretório

# Especifica o caminho para o arquivo Parquet de saída
output_parquet_path = './BTCUSDT-Dataset00.parquet'  # Substitua pelo seu caminho de saída desejado

# Etapa 1: Extrair e deletar arquivos ZIP de forma paralela
extract_and_delete(zip_dir_path)

# Etapa 2: Processar arquivos CSV de forma paralela e salvar como Parquet
process_csv_directory(zip_dir_path, output_parquet_path)

Iniciando a extração de 3 arquivos ZIP...
Extração concluída: BTCUSDT-trades-2024-10.zip
Extração concluída: BTCUSDT-trades-2024-11.zip
Extração concluída: BTCUSDT-trades-2024-12.zip
Extração de arquivos ZIP concluída.
Iniciando o processamento de 3 arquivos CSV...
Dados do arquivo BTCUSDT-trades-2024-10.csv processados.
Dados do arquivo BTCUSDT-trades-2024-11.csv processados.
Dados do arquivo BTCUSDT-trades-2024-12.csv processados.
Dados combinados salvos com sucesso em ./BTCUSDT-Dataset00.parquet
Processamento concluído.


# Particionamento do dataset em arquivos menores de no máximo 4GB

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa

# Caminho do arquivo original
input_file = "/Users/lordramos/Desktop/binance spot data/BTCUSDT-Dataset.parquet"

# Caminho para salvar os arquivos divididos
output_dir = "/Users/lordramos/Desktop/binance spot data/"

# Tamanho máximo de cada arquivo em bytes (4 GB = 4 * 1024^3 bytes)
max_size = 4 * 1024**3

# Inicialize as variáveis
part_number = 1
current_size = 0
output_file = f"{output_dir}BTCUSDT-Dataset-part-{part_number}.parquet"
writer = None

# Lê o arquivo original em pedaços
for batch in pq.ParquetFile(input_file).iter_batches(batch_size=10000):
    table = pa.Table.from_batches([batch])

    # Calcula o tamanho do batch atual
    batch_size = table.nbytes

    # Checa se o tamanho atual mais o novo batch excede o limite de 4 GB
    if current_size + batch_size > max_size:
        # Fecha o arquivo atual e inicia um novo arquivo
        if writer:
            writer.close()
        part_number += 1
        output_file = f"{output_dir}BTCUSDT-Dataset-part-{part_number}.parquet"
        current_size = 0  # Redefine o tamanho atual para o novo arquivo
        writer = None

    # Se o writer ainda não está definido, inicia um novo writer
    if writer is None:
        writer = pq.ParquetWriter(output_file, table.schema)

    # Escreve o batch no arquivo atual e atualiza o tamanho
    writer.write_table(table)
    current_size += batch_size

# Fecha o último arquivo
if writer:
    writer.close()

# Upload para HuggingFace

In [ ]:
from huggingface_hub import login, HfApi, HfFolder
import os

login(token="")

# Configurações de autenticação e detalhes do dataset
token = HfFolder.get_token()  # Assume que o token já está salvo localmente
api = HfApi()
dataset_id = "orion-research/btcusdt-spot-dataset"  # Substitua pelo nome do seu dataset

# Caminho da pasta onde os arquivos particionados estão salvos
output_dir = "/Users/lordramos/Desktop/binance spot data/"

# Cria o repositório no Hugging Face Hub (caso ainda não tenha sido criado)
api.create_repo(repo_id=dataset_id, repo_type="dataset", token=token, private=False)

# Loop para fazer o upload de cada arquivo particionado
for file_name in os.listdir(output_dir):
    if file_name.startswith("BTCUSDT-Dataset-part-") and file_name.endswith(".parquet"):
        file_path = os.path.join(output_dir, file_name)
        
        # Faz o upload do arquivo
        api.upload_file(
            path_or_fileobj=file_path,
            path_in_repo=file_name,
            repo_id=dataset_id,
            repo_type="dataset",
            token=token,
        )
        print(f"{file_name} upload completed.")
